<h2
    style="font-size:20px; font-family:'Helvetica Neue'" align="right"> Viktor Trokhymenko 
</h2>
<h3 
    style="font-size:16px; font-family:'Helvetica Neue'" align="right"> @vtrokhymenko
</h3>

In [3]:
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
np.random.seed(42)
import re
from pprint import pprint
import ast
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

---

### clean text

In [5]:
from nltk.corpus import stopwords
#pip install razdel
from razdel import tokenize # https://github.com/natasha/razdel
#pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [6]:
stopword_ru = stopwords.words('russian')
with open('../preprocessing/stopwords.txt', 'r', encoding='utf-8') as f:
    for w in f.readlines():
        stopword_ru.append(w)
print(len(stopword_ru))

252


In [7]:
def clean_text(text):
    
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')

    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    return text

---

### read data

In [9]:
%%time

df=pd.read_csv('data/news_lenta.csv')
print(df.shape)

(699777, 5)
CPU times: user 27.5 s, sys: 25.6 s, total: 53.2 s
Wall time: 1min 4s


In [10]:
df.head(2)

,tags,text,title,topic,url
0,Бокс и ММА,"Украинский боксер Александр Усик рассказал, чт...",Усик поделился планами на будущее после победы...,Спорт,https://lenta.ru/news/2018/07/22/usik_plany/
1,Политика,Израиль эвакуировал из Сирии активистов «Белых...,Израиль вывез из Сирии членов «Белых касок» по...,Мир,https://lenta.ru/news/2018/07/22/evacuation/


leave only [`url`,`text`] columns

In [11]:
df=df[['url','text']].copy()
df.head(2)

,url,text
0,https://lenta.ru/news/2018/07/22/usik_plany/,"Украинский боксер Александр Усик рассказал, чт..."
1,https://lenta.ru/news/2018/07/22/evacuation/,Израиль эвакуировал из Сирии активистов «Белых...


clean text

In [9]:
%%time
df['text_clean']=df.text.apply(lambda x: clean_text(x))

CPU times: user 6min 37s, sys: 20.5 s, total: 6min 58s
Wall time: 7min 27s


In [10]:
df.head()

,url,text,text_clean
0,https://lenta.ru/news/2018/07/22/usik_plany/,"Украинский боксер Александр Усик рассказал, чт...",украинский боксер александр усик рассказал что...
1,https://lenta.ru/news/2018/07/22/evacuation/,Израиль эвакуировал из Сирии активистов «Белых...,израиль эвакуировал из сирии активистов белых ...
2,https://lenta.ru/news/2018/07/22/hazard_real/,Лондонский «Челси» отказался продать мадридско...,лондонский челси отказался продать мадридскому...
3,https://lenta.ru/news/2018/07/22/otvet/,Вице-президент Федерации профессионального бок...,вицепрезидент федерации профессионального бокс...
4,https://lenta.ru/news/2018/07/22/moneyrich/,Журнал Billboard опубликовал рейтинг самых выс...,журнал billboard опубликовал рейтинг самых выс...


In [ ]:
%%time
df[['url','text_clean']].to_csv('data/news_lenta_clean.csv',index=False)

In [8]:
df = pd.read_csv('data/news_lenta_clean.csv')
print(df.shape)
df.head(2)

(699777, 2)


,url,text_clean
0,https://lenta.ru/news/2018/07/22/usik_plany/,украинский боксер александр усик рассказал что...
1,https://lenta.ru/news/2018/07/22/evacuation/,израиль эвакуировал из сирии активистов белых ...


---

### take our dataset with some names like `cleaned names`

In [10]:
cleaned_name = pd.read_csv('data/cleaned_names.csv')
print(cleaned_name.shape)
cleaned_name.head()

(2201, 5)


,cleaned_name,has_namesake,names( from text),cases,surname_cases
0,аба бакр аль-багдади,False,"Абу Бакр аль-Багдади,Абу Бакра Аль-Багдади,Абу...","аба бакр аль-багдади,абе бакр альбагдад,абой б...","аль-багдади, альбагдад"
1,абд раббо мансур хади,False,"Абд Раббо Мансур Хади,Абд Раббо Мансура Хади","абде раббо мансур хади, абдом раббо мансур ха...",хади
2,абдулла абдулла,False,"Абдулла Абдулла,Абдулле Абдулле,Абдуллой Абдул...","абдулла абдулла,абдулле абдулле,абдуллой абдул...","абдулла, абдулле, абдуллой, абдуллу"
3,абдулла гюль,False,"Абдулла Гюль,Абдулле Гюлю,Абдуллу Гюля,Абдуллы...","абдулла гюле,абдулла гюлем,абдулла гюль,абдулл...","гюле, гюлем, гюль, гюлю"
4,абубакар шекау,False,"Абубакар Шекау,Абубакара Шекау","абубакар шекау,абубакара шекау,абубакаре шекау...",шекау


In [11]:
cleaned_name = cleaned_name[['has_namesake','cleaned_name','cases']].copy()
#leave raws where mark has_namesake = False
#cleaned_name = cleaned_name[cleaned_name.has_namesake == 0]
print(cleaned_name.shape)
cleaned_name.head()

(2201, 3)


,has_namesake,cleaned_name,cases
0,False,аба бакр аль-багдади,"аба бакр аль-багдади,абе бакр альбагдад,абой б..."
1,False,абд раббо мансур хади,"абде раббо мансур хади, абдом раббо мансур ха..."
2,False,абдулла абдулла,"абдулла абдулла,абдулле абдулле,абдуллой абдул..."
3,False,абдулла гюль,"абдулла гюле,абдулла гюлем,абдулла гюль,абдулл..."
4,False,абубакар шекау,"абубакар шекау,абубакара шекау,абубакаре шекау..."


### edit cleaned_name
* leave only the 1st value if we have they a few
* join via `_` splitting `,`

In [12]:
cleaned_name['cleaned_name_new'] = cleaned_name.cleaned_name\
                                   .apply(lambda x: '_'.join(x.split(',')[0]\
                                                              .split(' ')))
cleaned_name.head()

,has_namesake,cleaned_name,cases,cleaned_name_new
0,False,аба бакр аль-багдади,"аба бакр аль-багдади,абе бакр альбагдад,абой б...",аба_бакр_аль-багдади
1,False,абд раббо мансур хади,"абде раббо мансур хади, абдом раббо мансур ха...",абд_раббо_мансур_хади
2,False,абдулла абдулла,"абдулла абдулла,абдулле абдулле,абдуллой абдул...",абдулла_абдулла
3,False,абдулла гюль,"абдулла гюле,абдулла гюлем,абдулла гюль,абдулл...",абдулла_гюль
4,False,абубакар шекау,"абубакар шекау,абубакара шекау,абубакаре шекау...",абубакар_шекау


### edit cases

create the list all possible mentions dropping whitespace name

drop null values

In [13]:
cleaned_name = cleaned_name[cleaned_name.cases.notnull()]

create list

In [14]:
cleaned_name['cases_new'] = cleaned_name.cases.apply(lambda x: [i.strip() for i in x.split(',')])
print(cleaned_name.shape)
cleaned_name.head(2)

(2201, 5)


,has_namesake,cleaned_name,cases,cleaned_name_new,cases_new
0,False,аба бакр аль-багдади,"аба бакр аль-багдади,абе бакр альбагдад,абой б...",аба_бакр_аль-багдади,"[аба бакр аль-багдади, абе бакр альбагдад, або..."
1,False,абд раббо мансур хади,"абде раббо мансур хади, абдом раббо мансур ха...",абд_раббо_мансур_хади,"[абде раббо мансур хади, абдом раббо мансур ха..."


---

## use flashtext to replace names

In [15]:
# https://github.com/vi3k6i5/flashtext
# !pip install flashtext
from flashtext import KeywordProcessor

create dict from our dataset where key is `cleaned_name_new` & values - `cases_new`

In [16]:
dict_cleaned_name={}
for _,i in cleaned_name.iterrows():
    dict_cleaned_name[f'{i.cleaned_name_new}']=i.cases_new
    
print(f'len dict_cleaned_name: {len(dict_cleaned_name)}')

len dict_cleaned_name: 2201


In [22]:
print(dict_cleaned_name['владимир_путин'])

['владимир путин', 'владимира путина', 'владимире путине', 'владимиром путиным', 'владимиру путину']


In [23]:
with open('data/cleaned_name.pickle', 'wb') as handle:
    pickle.dump(dict_cleaned_name, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
with open('data/cleaned_name.pickle', 'rb') as handle:
    dict_cleaned_name = pickle.load(handle)
    
print(dict_cleaned_name['владимир_путин'])

['владимир путин', 'владимира путина', 'владимире путине', 'владимиром путиным', 'владимиру путину']


create KeywordProcessor

In [25]:
#del kp
kp = KeywordProcessor()

kp.add_keywords_from_dict(dict_cleaned_name)
len(kp)

10510

In [30]:
text = '''когда дмитрий песков увидел владимира путина и авраама линкольна то подумал о иосифе сталине'''
print(kp.extract_keywords(text))
print(kp.replace_keywords(text))

['дмитрий_песков', 'владимир_путин', 'авраам_линкольн', 'иосиф_сталин']
когда дмитрий_песков увидел владимир_путин и авраам_линкольн то подумал о иосиф_сталин


### replace words in our dataset

In [31]:
df.head(2)

,url,text_clean
0,https://lenta.ru/news/2018/07/22/usik_plany/,украинский боксер александр усик рассказал что...
1,https://lenta.ru/news/2018/07/22/evacuation/,израиль эвакуировал из сирии активистов белых ...


In [499]:
%%time
df['text_clean_name'] = df.text_clean.apply(lambda x: kp.replace_keywords(x))

CPU times: user 28min 37s, sys: 31 s, total: 29min 8s
Wall time: 29min 46s


In [500]:
df.text_clean.head(10)

0    украинский боксер александр усик рассказал что...
1    израиль эвакуировал из сирии активистов белых ...
2    лондонский челси отказался продать мадридскому...
3    вицепрезидент федерации профессионального бокс...
4    журнал billboard опубликовал рейтинг самых выс...
5    бывшего участника сборной команды актобе квн д...
6    великобритания откажется от выплат отступных е...
7    талисманы олимпийских и паралимпийских игр в т...
8    около российской базы хмеймим в сирии произошл...
9    президент украины петр порошенко поздравил бок...
Name: text_clean, dtype: object

---

In [181]:
text = '''когда дмитрий песков увидел владимира путина и авраама линкольна то подумал о иосифе сталине'''

In [186]:
from cleanedNames import KeyProc

cn = KeyProc()

print(cn.extractKeywords(text))
print(cn.replaceKeywords(text))

['дмитрий_песков', 'владимир_путин', 'авраам_линкольн', 'иосиф_сталин']
когда дмитрий_песков увидел владимир_путин и авраам_линкольн то подумал о иосиф_сталин


In [185]:
df.head().text_clean.apply(lambda x: cn.replaceKeywords(x))

0    украинский боксер александр усик рассказал что...
1    израиль эвакуировал из сирии активистов белых ...
2    лондонский челси отказался продать мадридскому...
3    вицепрезидент федерации профессионального бокс...
4    журнал billboard опубликовал рейтинг самых выс...
Name: text_clean, dtype: object